In [59]:
#load fundamental library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #visualization


In [60]:
#load listinings data
dataset = pd.read_csv('../input/listings_summary.csv')

In [61]:
# Calculating distance using latitude and longitude
from geopy.distance import great_circle
def distance_from_berlin(lat,lon):
    center=(52.5027778, 13.404166666666667)
    record=(lat,lon)
    return great_circle(center,record).km
dataset['distance'] = dataset.apply(lambda x: distance_from_berlin(x.latitude, x.longitude), axis=1)

features=["distance","accommodates","bathrooms","bedrooms","beds","minimum_nights","amenities","guests_included","number_of_reviews","availability_365"]
dataset.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,distance
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",NaN,NaN,https://a0.muscache.com/im/pictures/260fd609-7...,NaN,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,96%,NaN,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4.0,4.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,...,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",NaN,$60.00,NaN,NaN,$200.00,$30.00,1,$28.00,4,1125,3 months ago,t,0,21,51,141,2018-11-07,118,2016-04-11,2018-10-28,93.0,10.0,9.0,10.0,10.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76,3.533182


In [62]:
dataset.amenities= dataset.amenities.str.count(',')
dataset.bathrooms= dataset.bathrooms.fillna(1)
dataset.bedrooms= dataset.bedrooms.fillna(1)
dataset.beds= dataset.beds.fillna(1)


In [63]:
# Categorical variables
dataset['room_type'] = pd.Categorical(dataset['room_type'])
# dataset['property_type'] = pd.Categorical(dataset['room_type'])
Dummies1 = pd.get_dummies(dataset['room_type'], prefix = 'category')
# Dummies2 = pd.get_dummies(dataset['property_type'], prefix = 'category')
# dataset2= pd.concat([Dummies1,Dummies2], axis=1)
X= pd.concat([Dummies1,dataset[features]], axis=1)

There is no null data in listining.csv. It is good news for us :)

In [64]:
dataset.price =  list(map(lambda x: float(str(x).replace(',','').replace('$','')),dataset.price)) 
y= dataset["price"]

In [65]:
dataset.price =  list(map(lambda x: float(str(x).replace(',','').replace('$','')),dataset.price)) 
y= dataset["price"]
# normalize data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

In [66]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
from sklearn.model_selection import GridSearchCV
param_grid = [{'max_depth': np.arange(1, 10),
              'min_samples_leaf': [1, 5, 10, 20, 50, 100],
              'min_weight_fraction_leaf': [0.0,0.1,0.3,0.5],
              'random_state':[1,3,4,7]}]
gridCV = GridSearchCV(estimator=regressor, param_grid=param_grid,cv=10)
gridCV = gridCV.fit(X_train, y_train)
print(gridCV.best_score_)
print(gridCV.best_params_)

0.5845468888596332
{'max_depth': 3, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'random_state': 1}


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [67]:
md = gridCV.best_params_["max_depth"]
msl  = gridCV.best_params_["min_samples_leaf"]
mwfl = gridCV.best_params_["min_weight_fraction_leaf"]
rs = gridCV.best_params_["random_state"]

In [68]:
regr_tree = DecisionTreeRegressor(max_depth=md,min_samples_leaf=msl,min_weight_fraction_leaf=mwfl,random_state=rs)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [69]:
#Measure error
from sklearn.metrics import mean_squared_error, r2_score
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {round(RMSE, 4)}")

r2 = r2_score(y_test, y_pred)
print(f"r2: {round(r2, 4)}")

RMSE: 67.2437
r2: 0.8656
